In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
from tqdm import tqdm, tqdm_notebook

from dataclasses import dataclass
from typing import *

from copy import deepcopy
from functools import reduce
from itertools import product

# `funcy` provides some convenient functionally-flavored combinators for manipulating collections and 
# dictionaries used throughout: https://funcy.readthedocs.io/en/stable/index.html
from funcy import * 

# from z3 import *
import z3

import igraph as ig
#import pygraphviz as pgv
from IPython.display import Image

In [3]:
# def pi(i: int):
#   assert i > -1
#   def projection(seq):
#     assert len(seq) > i
#     return seq[i]
#   return projection

# def p0(seq):
#   return pi(0)(seq)

# def p1(seq):
#   return pi(1)(seq)

# def p2(seq):
#   return pi(2)(seq)

# def fst(seq):
#   return p0(seq)

# def snd(seq):
#   return p1(seq)

# def thrd(seq):
#   return p2(seq)

In [4]:
def block_current_model(s):
  assert s.check() == z3.sat, f"no model to block!"
  m = s.model()
  if len([f for f in m.decls() if f.arity() == 0]) > 0:
    s.add(z3.Or([ f() != m[f] for f in m.decls() if f.arity() == 0]))
  return s

def all_smt(s):
  models = []
  while z3.sat == s.check():
    models.append(s.model())
    print(f"model # {len(models)}")
    #print(s.model())
    block_current_model(s)
  return models

def block_current_model_just(s, terms):
  assert s.check() == z3.sat, f"no model to block!"
  m = s.model()
  s.add(z3.Or([t != m.eval(t, model_completion=True) for t in terms]))
  return s

def all_smt_just(s, terms):
  models = []
  while z3.sat == s.check():
    models.append(s.model())
    print(f"model # {len(models)}")
    #print(s.model())
    block_current_model_just(s, terms)
  return models

In [5]:
from FV import *

# Tests

In [6]:
testM = 3
vals  = (-1, 0, 1)
allV3 = {FeatureVector.fromInts([a, b, c]) for a,b,c in zip(vals,vals,vals)}
v0    = FeatureVector.z(3)
v1    = FeatureVector.fromString("[0 0 +]")
v2    = FeatureVector.fromString("[+ 0 0]")
v3    = FeatureVector.fromString("[0 + +]")
v4    = FeatureVector.fromString("[+ + +]")
v5    = FeatureVector.fromString("[0 0 -]")
v6    = FeatureVector.fromString("[- - 0]")

In [7]:
assert all([v0 <= v
            for v in allV3])
assert all([v4 == v or (not v4 <= v)
            for v in allV3])
assert v0 <= v1 <= v3 <= v4
assert v2 <= v4
assert not (v5 <= v6)
assert not (v6 <= v5)
assert all([(not v.value[2].value == 1) or v1 <= v
            for v in allV3])
assert FeatureVector.meet(v2, v3) == v0
assert FeatureVector.meet(v1, v3) == v1
assert FeatureVector.join(v2, v3) == v4
assert v5 <= FeatureVector.join(v5, v6)

# Introduction

This notebook is a pedagogical version of an exploration of different possible SMT encodings in `z3` where each encoding will help us solve basic problems surrounding
 - searching for a "natural class" feature vector that describes a set of feature vectors ("observed sounds all thought to be participating in some phonological pattern").
 - searching for descriptions of rules consistent with a set of observed input-output examples.

## Intended audience

I will assume 
 - the reader has some level of familiarity with Python and introductory-level phonology
 - the reader understands (or can look at e.g. Wikipedia and understand) the difference between a partial order and a semilattice
 - the reader has been previously introduced to SAT/SMT solvers, but would like to see them applied to a phonological problem, and/or would like to see a key part of an SMT-solver driven workflow: exploring different encodings of a problem into combinations of representations and constraints understandable to the solver.

## Natural class problems

Suppose we have 
 - a language analyzed as having some inventory $\Sigma \subseteq \text{IPA}$
 - some choice of feature system $\mathcal{F} \subset \Sigma \times \{-1, 0, 1\}^m$ that we (harmlessly) assume associates every distinct symbol with a distinct $m$-length feature vector.
 
Then any time we have a set of observed symbols (or their synonymous feature vectors) $S \subseteq \Sigma$ that we think are all participating in some phonological pattern, what is a reasonable feature vector picking out the 'natural class' that definitely describes all of our observed symbol/vector types - but also no more other elements of $\Sigma$ than necessary?
Because most logically possible feature vectors either describe a real speech sound that happens to not be in $\Sigma$, or they describe no speech sound in any language at all, there in general are many feature vectors that describe any given subset of $\Sigma$.

In the specific context of this notebook, I am going to use the term "natural class problem" to refer to the problem of "finding the set of minimal feature vectors in $\mathcal{F}$ that describe as few other sounds of $\Sigma$ besides those in $S$".
 - **What else could 'natural class' mean, and what is left out of scope here?** In light of work like that of Jeff Mielke, a name for the specific notion of "natural class" that is relevant here would be something that indicates it is either or both a notion of natural class dependent on phonetically-driven classification of speech sounds, as opposed to one defined by some phonological environment, or simply defining a natural class as whatever participates in a significant pattern.

## SPE-style rewrite rule problems

fixme

## Organization of the notebook

fixme

# Define syntax of feature vectors

## Ternary-valued feature vectors with labeled features

$S \rightarrow [ (V \text{ } F)^* ]$

$V \rightarrow 0 | + | -$

$F \rightarrow voice | ant | strident | ...$

## Ternary-valued feature vectors with anonymous features

$S \rightarrow [ V^* ]$

$V \rightarrow 0 | + | -$

# `FV` Reference

## Basic types

A `FeatureVector` is a wrapper (a so-called "newtype") around a tuple of `FeatureValue`s.

A `FeatureValue` is a wrapper around an `int` $\in \{ -1, 0, +1 \}$.

In [8]:
FeatureVector.fromInts([0, -1, 1])
FeatureVector.fromString("[0 - +]")
FeatureVector.fromBooleans([False, True, True], [False, False, True])

FeatureVector(value=(FeatureValue(value=0), FeatureValue(value=-1), FeatureValue(value=1)))

FeatureVector(value=(FeatureValue(value=0), FeatureValue(value=-1), FeatureValue(value=1)))

FeatureVector(value=[FeatureValue(value=0), FeatureValue(value=-1), FeatureValue(value=1)])

In [9]:
v0 = FeatureVector.z(3)
v1 = FeatureVector.fromString("[0 0 +]")
v2 = FeatureVector.fromString("[0 0 -]")
v3 = FeatureVector.fromString("[0 + +]")
V  = (v0, v1, v2, v3)
for each in V:
  print(each)

[0 0 0]
[0 0 +]
[0 0 -]
[0 + +]


In [10]:
R = tuple([FeatureVector.r(3) for each in range(4)])
for each in R:
  print(each)

[+ - -]
[0 + -]
[0 0 -]
[- + 0]


## Basic operations

### Comparison

 - Feature vectors can be compared for equality.
 - Feature vectors can (sometimes) be compared for "specification".
 
### Manipulation

 - A feature vector can have its specified values flipped or erased.
 - For appropriate pairs of feature vectors, the operations of difference, meet, join, and (right) priority union are defined.

In [11]:
for left in V:
  for right in V:
    print(f"{left} <= {right} = {left <= right}")

[0 0 0] <= [0 0 0] = True
[0 0 0] <= [0 0 +] = True
[0 0 0] <= [0 0 -] = True
[0 0 0] <= [0 + +] = True
[0 0 +] <= [0 0 0] = False
[0 0 +] <= [0 0 +] = True
[0 0 +] <= [0 0 -] = False
[0 0 +] <= [0 + +] = True
[0 0 -] <= [0 0 0] = False
[0 0 -] <= [0 0 +] = False
[0 0 -] <= [0 0 -] = True
[0 0 -] <= [0 + +] = False
[0 + +] <= [0 0 0] = False
[0 + +] <= [0 0 +] = False
[0 + +] <= [0 0 -] = False
[0 + +] <= [0 + +] = True


In [12]:
print(v1)
print(FeatureVector.flip(v1))
print(~v1)
print(FeatureVector.erase(v1))

[0 0 +]
[0 0 -]
[0 0 -]
[0 0 0]


In [13]:
for left in [v3]:
  for right in [v0, v1, v2]:
    print(f"{left} - {right} = {left - right}")

[0 + +] - [0 0 0] = [0 + +]
[0 + +] - [0 0 +] = [0 + 0]
[0 + +] - [0 0 -] = [0 + +]


In [14]:
# (right) priority union
for left in [v3]:
  for right in [v0, v1, v2]:
    print(f"{left} >> {right} = {left >> right}")

[0 + +] >> [0 0 0] = [0 + +]
[0 + +] >> [0 0 +] = [0 + +]
[0 + +] >> [0 0 -] = [0 + -]


In [15]:
# meet
for left in [v3]:
  for right in [v0, v1, v2]:
    print(f"{left} & {right} = {left & right}")

[0 + +] & [0 0 0] = [0 0 0]
[0 + +] & [0 0 +] = [0 0 +]
[0 + +] & [0 0 -] = [0 0 0]


In [16]:
# join 
for left in [v3]:
  for right in [v0, v1]:
    print(f"{left} | {right} = {left | right}")

[0 + +] | [0 0 0] = [0 + +]
[0 + +] | [0 0 +] = [0 + +]


In [17]:
# not all joins are defined!
print(v1)
print(v2)
# v1 | v2 # raises AssertionError

[0 0 +]
[0 0 -]


## Details: The closed nearlattice structure on feature vectors

The set of all logically possible feature vectors of a given length form a special kind of partial order - a *bounded meet semilattice* where every pair of elements with an upper bound has a unique least upper bound ("join").

 - The minimal element of the semilattice ("zero", "the zero vector", "the glb") is the feature vector specified nowhere.
 - The set of maximal elements are all and only the fully-specified feature vectors.
 - The greatest lower bound ("glb", "meet", "$\land$") of two feature vectors $a,b$ is the feature vector with an intersection of their specified feature assignments.
   - The zero vector $z$ is the absorbing element of meet: $\forall a, a \land z = z = z \land a$
 - For two feature vectors that have *zero conflicting feature specifications*, the 'join' operation ("least upper bound", "lub", "$\lor$") is defined.
 - The join $c = a \lor b$ is the vector containing all feature specifications present in BOTH $a$ and $b$.

In [18]:
for l in [v0]:
  for r in [v0, v1, v2 ,v3]:
    print(showBinOp(f"{l}", "<=", f"{r}", l.__le__(r)))

[0 0 0] <= [0 0 0] = True
[0 0 0] <= [0 0 +] = True
[0 0 0] <= [0 0 -] = True
[0 0 0] <= [0 + +] = True


In [19]:
for l,r in [(v1, v2), (v1, v3)]:
  print(showBinOp(f"{l}", "&", f"{r}", FeatureVector.meet(l, r)))

[0 0 +] & [0 0 -] = [0 0 0]
[0 0 +] & [0 + +] = [0 0 +]


 - NB: The $\leq$ relation is NOT about the NUMBER of specified features.

In [20]:
# See how [0 0 +] and [0 0 -] are *incomparable* - neither is <= the other.
for l in [v1,v2]:
  for r in [v0, v1, v2 ,v3]:
    print(showBinOp(f"{l}", "<=", f"{r}", l.__le__(r)))

[0 0 +] <= [0 0 0] = False
[0 0 +] <= [0 0 +] = True
[0 0 +] <= [0 0 -] = False
[0 0 +] <= [0 + +] = True
[0 0 -] <= [0 0 0] = False
[0 0 -] <= [0 0 +] = False
[0 0 -] <= [0 0 -] = True
[0 0 -] <= [0 + +] = False


In [21]:
print(showBinOp(f"{v1}", "|", f"{v0}", f"{v1 | v0}"))
print(showBinOp(f"{v1}", "|", f"{v3}", f"{v1 | v3}"))

[0 0 +] | [0 0 0] = [0 0 +]
[0 0 +] | [0 + +] = [0 + +]


In [22]:
print(v1)
print(v2)
FeatureVector.comparable(v1, v2)
FeatureVector.joinable(v1, v2)
#v1 | v2  # throws assertion error because they have conflicting assignments

[0 0 +]
[0 0 -]


False

False

In [23]:
#fixme use iGraph to show the whole nearlattice for M=3 features

## Details: Priority union and its inverse

## Unconditioned, length-preserving rewrite rules

There is also a type in `FV` to represent total functions on the nearlattice that behave like (length-preserving, unconditioned) SPE-style rewrite rules:
$$T \longrightarrow C$$
where 
 - the target feature vector $T$ picks out those feature vectors that the rule can non-vacuously apply to ("preconditions").
 - the structural change vector $C$ describes what must be true after the rule has applied ("postconditions").
   - Note that we assume (this is a "frame axiom") anything unspecified by $C$ is unchanged by the rule.

In [24]:
r0 = FVMap.ULPRule(FeatureVector.fromString("[0 - 0]"), FeatureVector.fromString("[0 + +]"))
print(r0)
for each in (FeatureVector.fromString("[- - -]"),
             FeatureVector.fromString("[0 - +]"),
             FeatureVector.fromString("[- + -]")):
  print(f"r0({each}) = {r0(each)}")

[0 - 0] -> [0 + +]
r0([- - -]) = [- + +]
r0([0 - +]) = [0 + +]
r0([- + -]) = [- + -]


`FV` supports a number of queries about a given rule:

In [25]:
print(v1)
print(r0)
print(showApp("r0", f"{v1}", f"{r0(v1)}"))
FVMap.inDom(v1, r0)
FVMap.vacuous(v1, r0)
FVMap.consistent(r0, v1, v1)
FVMap.isFixpoint(v1, r0)
FVMap.isMutationpoint(v1, r0)

[0 0 +]
[0 - 0] -> [0 + +]
r0([0 0 +]) = [0 0 +]


False

True

True

True

False

# Define semantics of feature vectors - an *inventory* of objects

We use feature vectors to pick out a subset of a universe (set) of objects $O$. (I could more or less just as easily have chosen $\Sigma$ instead of $O$.)
We assume every object has exactly one associated feature vector and identify each object with its feature vector.

In general, the 'meaning' of a vector is the subset of $O$ that conform to its feature specification. 
Relative to the ordering introduced above, the meaning of $v$ is synonymous with its *upper closure*: 
$$[v] = \uparrow v = \{ v' | v \leq v' \}$$

The first step towards an exact solution to the natural class problem is to recognize that 

   - every subset $S$ of feature vectors has a (unique) greatest lower bound, viz. the meet of $S$:$$s_1 \land \ldots \land s_i \land \ldots \land s_n$$

In other words, the meet of $S$

  $$s_1 \land \ldots \land s_i \land \ldots \land s_n$$ 

is 
 - the **maximally specified** (= most verbose) **feature vector** 
 - ...whose interpretation contains **the minimum of other feature vectors besides those in $S$**.
 
The significance of this is that the meet of $S$ is the most verbose (syntactically worst) feature vector that has the best possible semantics we can get relative to what we want.

 - A plus is that we can compute the meet directly from the elements of $S$ - i.e. without any recourse to knowing exactly which subset of logically possible feature vectors describe actual objects.
 - Another plus is that the result of taking the meet of $S$ is another feature vector, and the interpretation of this single vector is a projection of a volume (an upper closure) of the nearlattice.
 - Both of these are clues we have a problem formulation amenable to a *version space algebra* approach, as elaborated on later/elsewhere.

With a succinct way of calculating an upper bound out of the way, the next question is - what does the rest of the solution set look like? What's in it? Does it always have a nice compact description?

In [26]:
M = 3
MAX_OBJECT_SET_SIZE = 26
# M = 10
# MAX_OBJECT_SET_SIZE = 100
MAX_SAMPLE_SIZE = 8
# MAX_OBJECT_SET_SIZE = MAX_NUM_VECTORS
# MAX_SAMPLE_SIZE = MAX_NUM_VECTORS 
MAX_NUM_VECTORS = 3 ** M
N = randint(2, MAX_OBJECT_SET_SIZE)
assert N <= (3 ** M)
print(f"|features| = {M}")
print(f"=> |logically possible vectors| = {MAX_NUM_VECTORS}")
print(f"max |objects = inventory| = {MAX_OBJECT_SET_SIZE}")
print(f"max |sample = hypothetical natural class| = {MAX_SAMPLE_SIZE}")
print(f"actual |objects = inventory| = {N}")

|features| = 3
=> |logically possible vectors| = 27
max |objects = inventory| = 26
max |sample = hypothetical natural class| = 8
actual |objects = inventory| = 21


In [31]:
O = FeatureVector.randomObjects(numFeatures=M, numObjects=N)

if len(O) < 30:
  for each in O:
    print(each)

[0 + 0]
[+ 0 0]
[0 0 -]
[0 0 +]
[0 - -]
[- 0 +]
[0 - +]
[- 0 -]
[+ - 0]
[- - 0]
[0 + -]
[+ 0 +]
[+ + 0]
[+ 0 -]
[- + 0]
[- + -]
[- + +]
[+ - +]
[+ - -]
[- - -]
[+ + -]


In [35]:
# S = randomNonemptySubset(set(O), 3)
S = randomInterestingSubset(400000, set(O), cap=MAX_SAMPLE_SIZE, minSpec=2) # "interesting" = the natural class is something other the trivial zero vector [0 ... ]

if S is not None:
  for each in S:
    print(each)

  print("---")
  print(grandFoldl(FeatureVector.meet)(S))
else:
  print("S is None.")

[+ + -]
[+ 0 -]
[+ - -]
---
[+ 0 -]


In [36]:
for each in range(5):
  myZero  = FeatureVector.z(M)
  rv      = FeatureVector.r(M)
  rv_meet = FeatureVector.meet(myZero, rv)
  print(f"{myZero} & {rv} = {rv_meet}")

[0 0 0] & [0 + -] = [0 0 0]
[0 0 0] & [+ - -] = [0 0 0]
[0 0 0] & [0 0 -] = [0 0 0]
[0 0 0] & [0 + 0] = [0 0 0]
[0 0 0] & [+ 0 +] = [0 0 0]


# The natural class problem with symbolic feature vectors

The natural class problem is not difficult per se - especially once we know the answer and that the answer (for positive data anyway) has a nice solution that only requires us to manipulate the representations of the feature vectors in the dataset in front of us.

In no small part because it *is* a simple problem that we already know the answer to - and because it is scaffolding for a closely related problem that is not as simple, we walk through different ways of encoding feature vectors and the constraints on them that reflect a "natural class" problem here, roughly in order of what I understand their complexity to be.

## EUF

(FIXME link to K&S 2016 chapter.)

In this setting, we start with EUF because it is "the simplest" theory on top of a SAT solver, and because what it makes difficult will motivate adding something new.

In [37]:
def naturalClassFor_euf(featureVectors, solver=None):
  if solver is None:
    solver = z3.Solver()
  
  data = tuple(sorted(featureVectors, key=FeatureVector.sortKey))
  M    = len(data[0])
  
  defined = [z3.Bool(f"def_{i}") for i in range(M)]
  valence = [z3.Bool(f"val_{i}") for i in range(M)]
  
  for datum in data:
    solver.add(describes_euf(defined, valence, datum))
  
  if solver.check() == z3.sat:    
    print("Found model.")
    m = solver.model()
    return solver, m
  else:
    print("No model found.")
    return solver
  

def describes_euf(defined, valence, featureVector):
  # \forall i, defined[i] -> valence[i] == featureVector.value[i].valence()
  return z3.And([z3.Or([z3.Not(d), v == x.valence()])
                 for d,v,x in zip(defined, valence, featureVector.value)])
  

In [38]:
for each in S:
  print(each)

[+ + -]
[+ 0 -]
[+ - -]


In [39]:
naturalClassFor_euf(S)

Found model.


([And(Or(Not(def_0), val_0 == True),
      Or(Not(def_1), False),
      Or(Not(def_2), val_2 == False)),
  And(Or(Not(def_0), val_0 == True),
      Or(Not(def_1), val_1 == True),
      Or(Not(def_2), val_2 == False)),
  And(Or(Not(def_0), val_0 == True),
      Or(Not(def_1), val_1 == False),
      Or(Not(def_2), val_2 == False))],
 [def_0 = False,
  def_1 = False,
  val_0 = False,
  def_2 = False,
  val_2 = False,
  val_1 = False])

We got exactly what we asked for, but not what we wanted: this is a description of the zero vector, and it does describe every feature vector in our subset of objects $S$ - because it describes every logically possible feature vector!

To capture `natural class`ness, we need to have/add constraints that either
 1. compute a natural class directly
 2. express the meaning of what a natural class is by describing them as some kind of maximal or minimal element among the set of feature vectors that describe $S$.
 
The first option is possible, but redundant and wasteful - unless we have some other motivation for "computing" natural classes symbolically. (We very plausibly do, but we'll get to that.)

The second option is also possible within EUF but (as far as I can tell) would be a tedious exercise in an adhoc explicit encoding of different combinations of propositions that correspond to counting cardinalities and doing comparisons. (See e.g. [answers to this question](https://cs.stackexchange.com/questions/13188/encoding-1-out-of-n-constraint-for-sat-solvers) for material on what "exactly $K$ of $N$ variables are true" encodings can be like.)

E.g. to revise the above constraints to define "a natural class feature vector for $S$" as not only
 - a feature vector that describes ("is consistent with") each elemnt of $S$
 
but also as 
 - a feature vector whose number of features used is maximal among those meeting other criteria
 
then two *more natural* approaches to take (involving the theory of linear arithmetic (???)) are:
 1. directly use the optimization functionality of z3 to find a symbolic feature vector consistent with the data and with maximal degree of specification (= maximal number of true values in the `defined` vector)
 2. start each search for a natural class feature vector at the maximum possible degree of specification it makes sense to start at, look for a solution, and if you can't find one, decrement the required degree of specification and try again
 
Each requires some way of reasoning about the specifcation level of a symbolic feature vector. 
In the absence of symbolic numeric variables, I don't think #1 is an option (or I can't imagine why it would be useful if it were).
To pursue option #2 without symbolic numeric variables, you need a set of mutually exclusive propositional variables, one for each possible length of feature vector (up to the context specific maximum m). 
Each of these propositions would be a disjunction over configurations of the symbolic feature vector corresponding to that length.

Because this option is *technically possible* but tedious and leaves a lot of programming work to be done and a lot of barriers to quickly making use of the results, we'll leave this thread behind.

## EUF + Boolean cardinality constraints

See [this](http://theory.stanford.edu/~nikolaj/programmingz3.html#sec-boolean-theories) for reference.

In [40]:
# Essentially a symbolic constructor for a FeatureVector
def mk_FVBooleans(numFeatures: int, identifier: str):
  return ([z3.Bool(f"def_{identifier}_{j}") for j in range(numFeatures)],
          [z3.Bool(f"val_{identifier}_{j}") for j in range(numFeatures)])
 
def mk_VBooleans(n: int, identifier: str):
  return [z3.Bool(f"{identifier}_{i}") for i in range(n)]

# Takes a model containing valuations for the symbolic booleans in defined and valence
# and returns two tuples of concrete Booleans that define a unique FeatureVector
def extractFeatureVectorFromModel_asBooleans(model, defined, valence):
  return (tuple(model[defined[i]] for i in range(len(defined))),
          tuple(model[valence[i]] for i in range(len(valence))))

# Takes a model containing valuations for the symbolic booleans in defined and valence
# and returns the corresponding concrete FeatureVector
def extractFeatureVectorFromModel(model, defined, valence):
  concrete_defined, concrete_valence = extractFeatureVectorFromModel_asBooleans(model, defined, valence)
  return FeatureVector.fromBooleans(concrete_defined, concrete_valence)

In [73]:
# def showBinGraphTest(op, a, b, cActual, cExpected):
#   return (f"{a} {op} {b} = {cActual} vs. {cExpected}")

# def testTotalBinFunc_Booleans(binPred, binOp, numFeatures, testSet=None):
#   s = z3.Solver()
  
#   def_l   = [z3.Bool(f"def_l_{j}") for j in range(numFeatures)]
#   val_l   = [z3.Bool(f"val_l_{j}") for j in range(numFeatures)]
#   def_r   = [z3.Bool(f"def_r_{j}") for j in range(numFeatures)]
#   val_r   = [z3.Bool(f"val_r_{j}") for j in range(numFeatures)]
#   def_res = [z3.Bool(f"res_{j}")   for j in range(numFeatures)]
#   val_res = [z3.Bool(f"res_{j}")   for j in range(numFeatures)]
  
#   #assert that binPred(def_l, val_l, def_r, val_r) == res
#   s.add(binPred(def_l, val_l, def_r, val_r) == )
#   def test(defined_left, valence_left, defined_right, valence_right)

# def testPartialBinOp_Booleans(binOpRel, binOp, numFratures, testSet=None):
#   pass #do I need to write this, or can I rewrite what I have below?

def testBinOp_Booleans(binOpRel, binOp, numFeatures, testSet=None, exceptions=False, none=False):
  if testSet is None:
    V = {FeatureVector.fromInts(x) for x in product((-1, 0, 1), repeat=numFeatures)}
  else:
    V = testSet

  s = z3.Solver()
  
  def_l  , val_l   = mk_FVBooleans(numFeatures, "l")
  def_r  , val_r   = mk_FVBooleans(numFeatures, "r")
  def_res, val_res = mk_FVBooleans(numFeatures, "res")
#   def_l   = [z3.Bool(f"def_l_{j}") for j in range(numFeatures)]
#   val_l   = [z3.Bool(f"val_l_{j}") for j in range(numFeatures)]
#   def_r   = [z3.Bool(f"def_r_{j}") for j in range(numFeatures)]
#   val_r   = [z3.Bool(f"val_r_{j}") for j in range(numFeatures)]
#   def_res = [z3.Bool(f"res_{j}")   for j in range(numFeatures)]
#   val_res = [z3.Bool(f"res_{j}")   for j in range(numFeatures)]  
  
  counterexamples = []
  for a, b in product(V,V):
    if exceptions:
      try:
        c = binOp(a,b)
      except:
        pass
      else:
        if none and c is None:
          pass
        else:
          s.push()
          s.add(eq_Booleans(def_l  , val_l  , *a.toBooleans()))
          s.add(eq_Booleans(def_r  , val_r  , *b.toBooleans()))

          s.add(binOpRel(def_l, val_l, def_r, val_r, def_res, val_res))
          if s.check() == z3.sat:
            res_symbolic = extractFeatureVectorFromModel(s.model(), def_res, val_res)
            if res_symbolic != c:
              counterexamples.append((a, b, c, res_symbolic))
          s.pop()
    else:
      c = binOp(a,b)
      if none and c is None:
        pass
      else:
        s.push()
        s.add(eq_Booleans(def_l  , val_l  , *a.toBooleans()))
        s.add(eq_Booleans(def_r  , val_r  , *b.toBooleans()))

        s.add(binOpRel(def_l, val_l, def_r, val_r, def_res, val_res))
        if s.check() == z3.sat:
          res_symbolic = extractFeatureVectorFromModel(s.model(), def_res, val_res)
          if res_symbolic != c:
            counterexamples.append((a, b, c, res_symbolic))
        s.pop()
  return counterexamples


def testStackOp_Booleans(opRel, op, numFeatures, testSet=None):
  s = z3.Solver()
  
  if testSet is None:
    V = {FeatureVector.fromInts(x) for x in product((-1, 0, 1), repeat=numFeatures)}
    numVectors = len(V)
#   elif testSet is "symbolic":
#     # FIXME - check on uninterpreted function symbols - are they the only vehicle for "testing" symbolic data against concrete functions?
    
#     #define numVectors 
    
#     stack = [mk_FVBooleans(numFeatures, f"V{k}") for k in range(numVectors)] 
#     defineds, valences = lmap(lambda pair: pair[0], stack), lmap(lambda pair: pair[1], stack)

#     #distinctness assertion

#    #
  else:
    V = testSet
    numVectors = len(V)

  result = op(V)

  stack = [mk_FVBooleans(numFeatures, f"V{k}") for k in range(numVectors)] 
  defineds, valences = lmap(lambda pair: pair[0], stack), lmap(lambda pair: pair[1], stack)
  for def_v, val_v, v in zip(defineds, valences, V):
    s.add(eq_Booleans(def_v, val_v, *v.toBooleans()))
  
  counterexamples = []
  
  defined_result, valence_result = mk_FVBooleans(numFeatures, f"res")  
  s.add(opRel(defineds, valences, defined_result, valence_result))
  
  if s.check() == z3.sat:
    result_symbolic = extractFeatureVectorFromModel(s.model(), defined_result, valence_result)
    if result_symbolic != result:
      counterexamples.append((V, result, result_symbolic))
  else:
    print("unsat!?")
  return counterexamples

In [74]:
def isSubseteq_Booleans(subsetVectorLeft, subsetVectorRight):
  assert len(subsetVectorLeft) == len(subsetVectorRight)
  left, right = subsetVectorLeft, subsetVectorRight
  return z3.And([z3.Implies(l, l == r)
                 for l,r in zip(left, right)])

def eq_Booleans(defined_left, valence_left, defined_right, valence_right):
  assert len(defined_left)  == len(valence_left)
  assert len(defined_right) == len(valence_right)
  assert len(defined_left)  == len(defined_right)
  numFeatures = len(defined_left)
  # left == right iff for every feature j,
  #  left is defined at j iff right is defined at j
  #  if left  is defined at j, left and right have the same valence at j
  # (if right is defined at j, right and left have the same valence at j) 
  return z3.And([z3.And([defined_left[j] == defined_right[j],
                        z3.Implies(defined_left[j],
                                   valence_left[j] == valence_right[j]),
                        z3.Implies(defined_right[j],
                                   valence_right[j] == valence_left[j])])
                 for j in range(numFeatures)])

def eq_Booleans1(defineds, valences):
  assert len(defineds) == len(valences)
  numVectors  = len(defineds)
  numFeatures = len(defineds[0])
  
  return z3.And([z3.And([ defineds[0][j] == defineds[i][j]
                        , z3.Implies(defineds[0][j], valences[0][j] == valences[i][j])
                        , z3.Implies(defineds[i][j], valences[i][j] == valences[0][j])])
                 for i in range(numVectors) for j in range(numFeatures)])

def neq_Booleans(defined_left, valence_left, defined_right, valence_right):
  return distinct_Booleans(defined_left, valence_left, defined_right, valence_right)

def distinct_Booleans(defined_left, valence_left, defined_right, valence_right):
  return z3.Not(eq_Booleans(defined_left, valence_left, defined_right, valence_right))

def pairwise_neq_Booleans1(defineds, valences):
  return pairwise_distinct_Booleans1(defineds, valences)

def pairwise_distinct_Booleans1(defineds, valences):
  assert len(defineds) == len(valences)
  numVectors  = len(defineds)
  numFeatures = len(defineds[0])
  
  #the stack of vectors is pairwise distinct iff forall vectors i,k
  # Either
  #  i == k
  # or
  # for at least one feature j, either
  #    1. i is defined at j but k isn't (or vice versa)
  #    2. both i and k are defined at j and have different valences
  return z3.And([z3.Or([ z3.Or([          defineds[i][j] != defineds[k][j]
                               , z3.And([ defineds[i][j]
                                        , defineds[k][j]
                                        , valences[i][j] != valences[k][j]
                                        ])
                               ])
                        for j in range(numFeatures)])
                 for i,k in product(range(numVectors), range(numVectors))
                 if i != k
                ])
  

# def lt_Booleans(defined_left, valence_left, defined_right, valence_right):
#   pass

# def lt_Booleans1(defineds, valences, defined_lowerBound, valence_lowerBound):
#   pass

def le_Booleans(defined_left, valence_left, defined_right, valence_right):
  assert len(defined_left)  == len(valence_left)
  assert len(defined_right) == len(valence_right)
  assert len(defined_left)  == len(defined_right)
  numFeatures = len(defined_left)
  # left <= right iff for every feature j, one of the following holds:
  #  - both left and right are   defined at j and have the same valence
  #  - both left and right are undefined at j
  #  - left is *un*defined at j and right is *defined* at j
  # Recall: the unique minimal element is defined *nowhere*
  return z3.And([z3.Or([z3.And([defined_left[j], defined_right[j], valence_left[j] == valence_right[j]]),
                        z3.And(z3.Not(defined_left[j]), z3.Not(defined_right[j])),
                        z3.And(z3.Not(defined_left[j]), defined_right[j])])
                 for j in range(numFeatures)])

def le_Booleans1(defineds, valences, defined_lowerBound, valence_lowerBound):
  assert len(defineds) == len(valences)
  numVectors  = len(defineds)
  
  assert len(defined_lowerBound) == len(valence_lowerBound)
  assert len(defined_lowerBound) == len(defineds[0])
  numFeatures = len(defined_lowerBound)
  
  return z3.And([z3.Or([ z3.And([defined_lowerBound[j], defineds[i][j], valence_lowerBound[j] == valences[i][j]])
                       , z3.And(z3.Not(defined_lowerBound[j]), z3.Not(defineds[i][j]))
                       , z3.And(z3.Not(defined_lowerBound[j]), defineds[i][j])
                       ])
                 for i in range(numVectors) for j in range(numFeatures)])
  
def ge_Booleans(defined_left, valence_left, defined_right, valence_right):
  assert len(defined_left)  == len(valence_left)
  assert len(defined_right) == len(valence_right)
  assert len(defined_left)  == len(defined_right)
  return le_Booleans(defined_right, valence_right, defined_left, valence_left)
  
def ge_Booleans1(defineds, valences, defined_upperBound, valence_upperBound):
  assert len(defineds) == len(valences)
  numVectors  = len(defineds)
  
  assert len(defined_upperBound) == len(valence_upperBound)
  assert len(defined_upperBound) == len(defineds[0])
  numFeatures = len(defined_upperBound)
  
  return z3.And([z3.Or([ z3.And([defineds[i][j], defined_upperBound[j], valences[i][j] == valence_upperBound[j]])
                       , z3.And(z3.Not(defineds[i][j]), z3.Not(defined_upperBound[j]))
                       , z3.And(z3.Not(defineds[i][j]), defined_upperBound[j])
                       ])
                 for i in range(numVectors) for j in range(numFeatures)])

# def gt_Booleans(defined_left, valence_left, defined_right, valence_right):
#   pass

# def gt_Booleans1(defineds, valences, defined_upperBound, valence_upperBound):
#   pass

def comparable_Booleans(defined_left, valence_left, defined_right, valence_right):
  return z3.Or(le_Booleans( defined_left,  valence_left, defined_right, valence_right),
               le_Booleans(defined_right, valence_right,  defined_left,  valence_left))

#pairwise comparability
def comparable_Booleans1(defineds, valences):
  return z3.And([comparable_Booleans(a[0], a[1], b[0], b[1])
                 for a,b in product(zip(defineds, valences), 
                                    zip(defineds, valences))])

# def coveredBy_Booleans(defined_left, valence_right, defined_right, valence_right):
#   assert len(defined_left)  == len(valence_left)
#   assert len(defined_right) == len(valence_right)
#   assert len(defined_left)  == len(defined_right)
  
# def max_Booleans(defineds, valences, defined_max, valence_max):
#   assert len(defined_max) == len(valence_max)
#   assert len(defineds)    == len(valences)
#   assert len(defineds[0]) == len(defined_max)
#   numFeatures = len(defined_max)
#   numObjects = len(defineds)

# def min_Booleans(defineds, valences, defined_min, valence_min):
#   pass


def meet_Booleans(defined_left, valence_left, defined_right, valence_right, defined_meet, valence_meet):
  assert len(defined_left)  == len(valence_left)
  assert len(defined_right) == len(valence_right)
  assert len(defined_meet)  == len(valence_meet)
  assert len(defined_left)  == len(defined_right)
  assert len(defined_meet)  == len(defined_left)
  numFeatures = len(defined_left)
  # left \land right == meet iff for all features j
  #  def_meet[j] == def_left[j] \land def_right[j] \land val_left[j] == val_right[j]
  #  def_meet[j] -> val_meet[j] == val_left[j] \land val_meet[j] == val_right[j]
  return z3.And([z3.And([defined_meet[j] == z3.And([defined_left[j], defined_right[j], valence_left[j] == valence_right[j]]),
                         z3.Implies(defined_meet[j], z3.And([valence_meet[j] == valence_left[j],
                                                             valence_meet[j] == valence_right[j]]))])
                 for j in range(numFeatures)])

def meet_Booleans1(defineds, valences, defined_meet, valence_meet):
  assert len(defineds)      == len(valences)
  assert len(defined_meet)  == len(valence_meet)
  numObjects  = len(defineds)
  numFeatures = len(defined_meet)
  
  allDefinedAtJandTrue  = lambda j: z3.And([z3.And(defineds[i][j],        valences[i][j])  for i in range(numObjects)])
  allDefinedAtJandFalse = lambda j: z3.And([z3.And(defineds[i][j], z3.Not(valences[i][j])) for i in range(numObjects)])
  allDefinedAtJAndSame  = lambda j: z3.Or([allDefinedAtJandTrue(j), allDefinedAtJandFalse(j)])
  meetDefinedAtJ        = lambda j: allDefinedAtJAndSame(j)
  meetTrueAtJ           = lambda j: allDefinedAtJandTrue(j)
  meetFalseAtJ          = lambda j: allDefinedAtJandFalse(j)
  
  return z3.And([z3.And([ defined_meet[j] == meetDefinedAtJ(j)
                        , z3.Implies(meetTrueAtJ( j),        valence_meet[j] )
                        , z3.Implies(meetFalseAtJ(j), z3.Not(valence_meet[j]))
                        ])
                 for j in range(numFeatures)])

#   return z3.And([z3.And([defined_meet[j] == meetDefinedAtJ(j), 
#                          z3.Implies(defined_meet[j],        valence_meet[j]  == meetTrueAtJ(j)), 
#                          z3.Implies(defined_meet[j], z3.Not(valence_meet[j]) == meetFalseAtJ(j))])
#                  for j in range(numFeatures)])
  # m is the meet of V iff for all features j
  #  def_meet[j] == z3.And([defineds[i][j] for i in range(len(numObjects))])
  #  val_meet[j] == z3.And([def_meet[j]] + [])

def joinable_Booleans(defined_left, valence_left, defined_right, valence_right):
  assert len(defined_right) == len(valence_right)
  assert len(defined_left)  == len(defined_right)
  numFeatures = len(defined_left)
  # left \lor right exists iff for all features j
  #  Either
  #   1.      def_l[j] \land      def_r[j] \land val_l[j] == val_r[j]
  #   2. \neg def_l[j]
  #   3. \neg def_r[j]
  return z3.And([z3.Or([ z3.And([defined_left[j], defined_right[j], valence_left[j] == valence_right[j]])
                       , z3.Not(defined_left[j])
                       , z3.Not(defined_right[j])
                       ])
                 for j in range(numFeatures)])

def joinable_Booleans1(defineds, valences):
  assert len(defineds)      == len(valences)
#   assert len(defined_join)  == len(valence_join)
  numObjects  = len(defineds)
  numFeatures = len(defineds[0])
#   numFeatures = len(defined_join)
  
  atLeastOneDefinedAtJAndTrue  = lambda j: z3.Or([z3.And(        defineds[i][j]
                                                        ,        valences[i][j]
                                                        ) 
                                                  for i in range(numObjects)])
  atLeastOneDefinedAtJAndFalse = lambda j: z3.Or([z3.And(        defineds[i][j]
                                                        , z3.Not(valences[i][j])
                                                        )
                                                  for i in range(numObjects)])
  noConflictsAtJ               = lambda j: z3.Not(z3.And( atLeastOneDefinedAtJAndTrue(j)
                                                        , atLeastOneDefinedAtJAndFalse(j)
                                                        ))
  return z3.And([noConflictsAtJ(j)
                 for j in range(numFeatures)])
#   assert len(defineds)      == len(valences)
#   assert len(defined_meet)  == len(valence_meet)
#   numObjects  = len(defineds)
#   numFeatures = len(defined_meet)
  
#   exactlyOneDefinedAt
#   allDefinedAtJandTrue  = lambda j: z3.And([z3.And(defineds[i][j],        valences[i][j])  for i in range(numObjects)])
#   allDefinedAtJandFalse = lambda j: z3.And([z3.And(defineds[i][j], z3.Not(valences[i][j])) for i in range(numObjects)])
#   allDefinedAtJAndSame  = lambda j: z3.Or([allDefinedAtJandTrue(j), allDefinedAtJandFalse(j)])
#   foo = #FIXME THIS IS THE MEET1 DEFINITION
#   meetDefinedAtJ        = lambda j: allDefinedAtJAndSame(j)
#   meetTrueAtJ           = lambda j: allDefinedAtJandTrue(j)
#   meetFalseAtJ          = lambda j: allDefinedAtJandFalse(j)
  
#   return z3.And([z3.And([ defined_meet[j] == meetDefinedAtJ(j)
#                         , z3.Implies(meetTrueAtJ( j),        valence_meet[j] )
#                         , z3.Implies(meetFalseAtJ(j), z3.Not(valence_meet[j]))])
#                  for j in range(numFeatures)])
# #   return z3.And([z3.And([defined_meet[j] == meetDefinedAtJ(j), 
# #                          z3.Implies(defined_meet[j],        valence_meet[j]  == meetTrueAtJ(j)), 
# #                          z3.Implies(defined_meet[j], z3.Not(valence_meet[j]) == meetFalseAtJ(j))])
# #                  for j in range(numFeatures)])
#   # m is the meet of V iff for all features j
#   #  def_meet[j] == z3.And([defineds[i][j] for i in range(len(numObjects))])
#   #  val_meet[j] == z3.And([def_meet[j]] + [])

def join_Booleans(defined_left, valence_left, defined_right, valence_right, defined_join, valence_join):
  assert len(defined_right) == len(valence_right)
  assert len(defined_left)  == len(defined_right)
  numFeatures = len(defined_left)
  
  exactlyOneDefinedAtJ         = lambda j: z3.Xor(                  defined_left[j]
                                                 ,                 defined_right[j]
                                                 )
  exactlyOneDefinedAndTrueAtJ  = lambda j: z3.Xor(  z3.And(         defined_left[j]
                                                          ,         valence_left[j])
                                                 ,  z3.And(        defined_right[j]
                                                          ,        valence_right[j])
                                                 )
  exactlyOneDefinedAndFalseAtJ = lambda j: z3.Xor(  z3.And(        defined_left[j]
                                                          , z3.Not(valence_left[j]))
                                                 ,  z3.And(        defined_right[j]
                                                          , z3.Not(valence_right[j]))
                                                 )
  
  bothDefinedAndTrueAtJ        = lambda j: z3.And([        defined_left[j] ,        defined_right[j],        
                                                           valence_left[j] ,        valence_right[j]])
  bothDefinedAndFalseAtJ       = lambda j: z3.And([        defined_left[j] ,        defined_right[j], 
                                                    z3.Not(valence_left[j]), z3.Not(valence_right[j])])
  bothDefinedAndSameAtJ        = lambda j: z3.Or(bothDefinedAndTrueAtJ(j), bothDefinedAndFalseAtJ(j))
  
  # left \lor right == join iff joinable(left, right) and for all features j
  # def_join[j] == bothDefinedAndSameAtJ( j) \lor exactlyOneDefinedAtJ(j)
  # bothDefinedAndSameAtJ( j) = (def_left[j] \land def_right[j]) \land (val_left[j] == val_right[j])
  # exactlyOneDefinedAtJ
  # bothDefinedAndTrueAtJ( j) ->  val_join[j]
  # bothDefinedAndFalseAtJ(j) -> ~val_join[j]
  return z3.And([ joinable_Booleans(defined_left, valence_left, defined_right, valence_right) 
                ] +
                [z3.And([        defined_join[j]  == z3.Or([ bothDefinedAndSameAtJ(j)
                                                           , exactlyOneDefinedAtJ(j)])
#                         ,        valence_join[j]  == z3.Or( bothDefinedAndTrueAtJ(j)
#                                                           , exactlyOneDefinedAndTrueAtJ(j))
#                         , z3.Not(valence_join[j]) == z3.Or( bothDefinedAndFalseAtJ(j)
#                                                           , exactlyOneDefinedAndFalseAtJ(j))
                        , z3.Implies( z3.Or( bothDefinedAndTrueAtJ(j)
                                           , exactlyOneDefinedAndTrueAtJ(j))
                                    ,        valence_join[j]
                                    )
                        , z3.Implies( z3.Or( bothDefinedAndFalseAtJ(j)
                                           , exactlyOneDefinedAndFalseAtJ(j))
                                    , z3.Not(valence_join[j])
                                    )
                        ])
                 for j in range(numFeatures)])

def join_Booleans1(defineds, valences, defined_join, valence_join):
  assert len(defineds)      == len(valences)
  assert len(defined_join)  == len(valence_join)
  numObjects  = len(defineds)
  numFeatures = len(defined_join)
  
  atLeastOneDefinedAtJAndTrue  = lambda j: z3.Or([z3.And(        defineds[i][j]
                                                        ,        valences[i][j]
                                                        ) 
                                                  for i in range(numObjects)])
  atLeastOneDefinedAtJAndFalse = lambda j: z3.Or([z3.And(        defineds[i][j]
                                                        , z3.Not(valences[i][j])
                                                        )
                                                  for i in range(numObjects)])
  joinIsDefinedAtJ         = lambda j: z3.Xor( atLeastOneDefinedAtJAndTrue(j)
                                             , atLeastOneDefinedAtJAndFalse(j)
                                             )
  joinIsDefinedAtJAndTrue  = lambda j: z3.And( joinIsDefinedAtJ(j)
                                             , atLeastOneDefinedAtJAndTrue(j)
                                             )
  joinIsDefinedAtJAndFalse = lambda j: z3.And( joinIsDefinedAtJ(j)
                                             , atLeastOneDefinedAtJAndFalse(j)
                                             )
  return z3.And([z3.And([ joinable_Booleans1(defineds, valences)
                        , defined_join[j] == joinIsDefinedAtJ(j)
                        , z3.Implies( joinIsDefinedAtJAndTrue(j)
                                    , valence_join[j]
                                    )
                        , z3.Implies( joinIsDefinedAtJAndFalse(j)
                                    , z3.Not(valence_join[j])
                                    )
                        ])
                 for j in range(numFeatures)])


In [75]:
def findObjectSetWithNonUniqueMeet(numFeatures, numObjects):
  assert numFeatures > 0
  assert numObjects  > 0
  maxNumVectors = 3 ** numFeatures
  assert numObjects <= maxNumVectors, f"Max # distinct vectors for {numFeatures} features is {maxNumVectors}, but numObjects is '{numObjects}'"
  
  s = z3.Solver()
  
  stack = [mk_FVBooleans(numFeatures, f"V_{i}") for i in range(numObjects)]
  defineds    , valences     = lmap(lambda pair: pair[0], stack), lmap(lambda pair: pair[1], stack)
  s.add(pairwise_distinct_Booleans1(defineds, valences))
  
  defined_meet, valence_meet = mk_FVBooleans(numFeatures, "m")
  s.add(meet_Booleans1(defineds, valences, defined_meet, valence_meet))
  
  defined_alt , valence_alt  = mk_FVBooleans(numFeatures, "a")
  s.add(meet_Booleans1(defineds, valences, defined_alt , valence_alt))
  
  s.push()
  s.add(distinct_Booleans(defined_meet, valence_meet, defined_alt, valence_alt))

  if s.check() == z3.sat:
    print('Found model!?') # not expected!
    return "nonunique meets", s, s.model(), stack, (defined_meet, valence_meet), (defined_alt, valence_alt)
  else:
    print('No model with distinct second meet') # evidence our definition of meet_Booleans1 is correct if this is *always* the outcome
    s.pop()
    if s.check() == z3.sat:
      print('unique meet')
      return "unique meet/popped", s, s.model(), stack, (defined_meet, valence_meet), (defined_alt, valence_alt)
    print('No meet!')
    return "no meet", s

In [84]:
# Ms =  list(range(3,5))
# Ns = [list(range(1, 3 ** m)) for m in Ms]
# MNs = lmapcat(lambda pair: [(pair[0], each) for each in pair[1]], 
#               zip(Ms, Ns))
# meetIsUnique, meetIsNotUnique, noMeet = [], [], []
# for eachM, eachN in tqdm(MNs, total = len(MNs)):
#   print(f"m = {eachM}")
#   print(f"n = {eachN}")
#   res = findObjectSetWithNonUniqueMeet(eachM, eachN)
#   if res[0] == "nonunique meets":
#     meetIsNotUnique.append(res)
#   elif res[0] == "no meet":
#     noMeet.append(res)
#   elif res[0] == "unique meet/popped":
#     meetIsUnique.append(res)
# print(f"|models where meet is *not* unique| = {len(meetIsNotUnique)}") # expecting zero
# print(f"|models where no meet exists| = {len(noMeet)}") # expecting zero
# print(f"|models where meet is unique| = {len(meetIsUnique)}")

  3%|▎         | 3/106 [00:00<00:04, 25.34it/s]

m = 3
n = 1
No model with distinct second meet
unique meet
m = 3
n = 2
No model with distinct second meet
unique meet
m = 3
n = 3
No model with distinct second meet
unique meet
m = 3
n = 4
No model with distinct second meet
unique meet
m = 3
n = 5


  6%|▌         | 6/106 [00:00<00:04, 22.59it/s]

No model with distinct second meet
unique meet
m = 3
n = 6
No model with distinct second meet
unique meet
m = 3
n = 7
No model with distinct second meet
unique meet
m = 3
n = 8


  8%|▊         | 9/106 [00:00<00:06, 15.32it/s]

...

unique meet
m = 4
n = 79
No model with distinct second meet


 99%|█████████▉| 105/106 [04:07<00:08,  8.51s/it]

unique meet
m = 4
n = 80
No model with distinct second meet


100%|██████████| 106/106 [04:16<00:00,  2.42s/it]

unique meet
|models where meet is *not* unique| = 0
|models where no meet exists| = 0
|models where meet is unique| = 106


In [76]:
def findObjectSetWithNonUniqueJoin(numFeatures, numObjects):
  assert numFeatures > 0
  assert numObjects  > 0
  maxNumVectors = 3 ** numFeatures
  assert numObjects <= maxNumVectors, f"Max # distinct vectors for {numFeatures} features is {maxNumVectors}, but numObjects is '{numObjects}'"
  
  s = z3.Solver()
  
  stack = [mk_FVBooleans(numFeatures, f"V_{i}") for i in range(numObjects)]
  defineds    , valences     = lmap(lambda pair: pair[0], stack), lmap(lambda pair: pair[1], stack)
  s.add(pairwise_distinct_Booleans1(defineds, valences))
  
  defined_join, valence_join = mk_FVBooleans(numFeatures, "m")
  s.add(join_Booleans1(defineds, valences, defined_join, valence_join))
  
  defined_alt , valence_alt  = mk_FVBooleans(numFeatures, "a")
  s.add(join_Booleans1(defineds, valences, defined_alt , valence_alt))
  
  s.push()
  s.add(distinct_Booleans(defined_join, valence_join, defined_alt, valence_alt))

  if s.check() == z3.sat:
    print('Found model!?') # not expected!
    return "nonunique joins", s, s.model(), stack, (defined_join, valence_join), (defined_alt, valence_alt)
  else:
    print('No model with distinct second join') # evidence our definition of join_Booleans1 is correct if this is *always* the outcome
    s.pop()
    if s.check() == z3.sat:
      print('unique join')
      return "unique join/popped", s, s.model(), stack, (defined_join, valence_join), (defined_alt, valence_alt)
    print('No join!')
    return "no join", s

In [ ]:
# Ms =  list(range(3,5))
# Ns = [list(range(1, 3 ** m)) for m in Ms]
# MNs = lmapcat(lambda pair: [(pair[0], each) for each in pair[1]], 
#               zip(Ms, Ns))
# joinIsUnique, joinIsNotUnique, noJoin = [], [], []
# for eachM, eachN in tqdm(MNs, total = len(MNs)):
#   print(f"m = {eachM}")
#   print(f"n = {eachN}")
#   res = findObjectSetWithNonUniqueJoin(eachM, eachN)
#   if res[0] == "nonunique joins":
#     joinIsNotUnique.append(res)
#   elif res[0] == "no join":
#     noJoin.append(res)
#   elif res[0] == "unique join/popped":
#     joinIsUnique.append(res)
# print(f"|models where join is *not* unique| = {len(joinIsNotUnique)}") # expecting zero
# print(f"|models where no join exists| = {len(noJoin)}") # expecting *non*zero, perhaps most
# print(f"|models where join is unique| = {len(joinIsUnique)}")

  3%|▎         | 3/106 [00:00<00:03, 25.99it/s]

m = 3
n = 1
No model with distinct second join
unique join
m = 3
n = 2
No model with distinct second join
unique join
m = 3
n = 3
No model with distinct second join
unique join
m = 3
n = 4
No model with distinct second join
unique join
m = 3
n = 5


  6%|▌         | 6/106 [00:00<00:06, 15.17it/s]

No model with distinct second join
unique join
m = 3
n = 6
No model with distinct second join
unique join
m = 3
n = 7


  8%|▊         | 8/106 [00:00<00:08, 11.16it/s]

...

No model with distinct second join
unique join
m = 4
n = 16


 40%|███▉      | 42/106 [00:21<00:28,  2.27it/s]

No model with distinct second join
unique join
m = 4
n = 17
No model with distinct second join


 41%|████      | 43/106 [31:46<9:48:19, 560.32s/it]

No join!
m = 4
n = 18
No model with distinct second join


In [78]:
joinCounterexamples = testBinOp_Booleans(join_Booleans, FeatureVector.join, 3, exceptions=True)
len(joinCounterexamples)

for i,x in tqdm(enumerate(joinCounterexamples), total=len(joinCounterexamples)):
  print(f"{i}: " + showBinGraphTest("v", x[0], x[1], x[2], x[3]))

0

0it [00:00, ?it/s]


In [79]:
meetCounterexamples = testBinOp_Booleans(meet_Booleans, FeatureVector.meet, 3)
len(meetCounterexamples)

for i,x in tqdm(enumerate(meetCounterexamples)):
  print(f"{i}: " + showBinGraphTest("^", x[0], x[1], x[2], x[3]))

0

In [51]:
print(f"M = {M}")
print(f"|O| = {len(O)}")
print(f"2^|O| = {2 ** len(O)}")
print('===')

ssv_cxs = []
for idx, ssv in tqdm.tqdm(iterable=enumerate(allSubsetVectors(None)(len(O))), total=2 ** len(O)):
# for ssv in allSubsetVectors(None)(len(O)):
  ss_fromV         = mk_subset_fromVector(tuple)(O)(ssv)
  cxs = testStackOp_Booleans(meet_Booleans1, grandFoldl(FeatureVector.meet), M, testSet=ss_fromV)
  if len(cxs) > 0:
    print(f"{idx}: |ssv_cxs| = {len(ssv_cxs)}")
    ssv_cxs.append((ssv, ss_fromV, cxs))
  
#   print("---")
#   print(ssv)
#   print(ssv_fromSS_fromV)
#   print(ss_fromV)
  
#   if ssv_fromSS_fromV != ssv:
#     ssv_cxs.add((ssv, ssv_fromSS_fromV, ss_fromV))
  
len(ssv_cxs)
for ssv, ss_fromV, cxs in ssv_cxs:
  print('---')
  print(f"ssv:\n{ssv}")
  print(f"ss_fromV:")
  for idx, each in enumerate(ss_fromV):
    print(f"{idx}: {each}")
  print(f"cxs:")
  for theStack, actualResult, symbolicResult in cxs:
    print(f"concrete vs. symbolic: {actualResult} vs. {symbolicResult}")

  0%|          | 2/1048576 [00:00<17:38:32, 16.51it/s]

M = 3
|O| = 20
2^|O| = 1048576
===


 43%|████▎     | 448603/1048576 [4:31:58<6:03:44, 27.49it/s] 


KeyboardInterrupt: 

In [57]:
def findAmbiguousObjectSet(numFeatures, numObjects, solver=None):
  if solver is None:
    solver = z3.Solver()
  
  stack = [mk_FVBooleans(numFeatures, f"obj_{i}") for i in range(numObjects)]
  defineds, valences = lmap(fst, stack), lmap(snd, stack)
  solver.add(pairwise_distinct_Booleans1(defineds, valences))
  
  inSubset = mk_VBooleans(numObjects, "inSubset")
  
  meet = mk_FVBooleans(numFeatures, "meet")
  defined_meet, valence_meet = meet
  solver.add(meet_Booleans1(defineds, valences, defined_meet, valence_meet))
  
  # assert that every object in the subset must be in the interpretation of the meet (the converse does not in general hold)
  # assert that for every object i, i is in the interpretation of the meet iff the meet <= object i 
  meet_interp = mk_VBooleans(numObjects, "meet_interp")
  solver.add(z3.And([ z3.And( z3.Implies(inSubset[i], meet_interp[i])
                            , meet_interp[i] == le_Booleans(defined_meet, valence_meet, defineds[i], valences[i]))
                    for i in range(numObjects)]))
  
  alt = mk_FVBooleans(numFeatures, "alt")
  defined_alt, valence_alt = alt
  solver.add(distinct_Booleans(defined_meet, valence_meet, defined_alt, valence_meet))
  
  alt_interp = mk_VBooleans(numObjects, "alt_interp")
  solver.push()
  solver.add(z3.And([m == a 
                     for m,a in zip(meet_interp, 
                                    alt_interp)
                    ]))
  
  if solver.check() == z3.sat:
    print("Found vector distinct from meet with same interpretation")
    return "meet synonym", solver, solver.model(), stack, meet, meet_interp, alt, alt_interp
  else:
    solver.pop()
    if solver.check() == z3.sat:
      print("Only meet has its interpretation")
      return "unique meet", solver, solver.model(), stack, meet, meet_interp, alt, alt_interp
    else:
      print("Meet does not exist with current parameters")
      return "no meet", solver

In [59]:
Ms  = tuple(range(2,5))
Ns  = tuple([tuple(range(2, 3 ** m)) for m in Ms])
MNs = lmapcat(lambda pair: tuple([(pair[0], each) for each in pair[1]]),
              zip(Ms, Ns))
# MNs = lmapcat(lambda pair: [(pair[0], each) for each in pair[1]], 
#               zip(Ms, Ns))
meetSynonyms, uniqueMeets, noMeets = [], [], []
for eachM, eachN in tqdm(MNs, total = len(MNs)):
  print(f"m = {eachM}")
  print(f"n = {eachN}")
  res = findAmbiguousObjectSet(eachM, eachN)
  if res[0] == "meet synonym":
    meetSynonyms.append(res)
  elif res[0] == "unique meet":
    uniqueMeets.append(res)
  elif res[0] == "no meet":
    noMeets.append(res)
print(f"|models where meet has a synonym| = {len(meetSynonyms)}") # expecting non-zero
print(f"|models where meet is unique| = {len(uniqueMeets)}") # expecting most
print(f"|models where no meet exists| = {len(noMeets)}") # expecting zero

  3%|▎         | 3/111 [00:00<00:04, 24.03it/s]

m = 2
n = 2
Found vector distinct from meet with same interpretation
m = 2
n = 3
Found vector distinct from meet with same interpretation
m = 2
n = 4
Found vector distinct from meet with same interpretation
m = 2
n = 5
Found vector distinct from meet with same interpretation
m = 2
n = 6


  8%|▊         | 9/111 [00:00<00:04, 23.88it/s]

Found vector distinct from meet with same interpretation
m = 2
n = 7
Found vector distinct from meet with same interpretation
m = 2
n = 8
Found vector distinct from meet with same interpretation
m = 3
n = 2
Found vector distinct from meet with same interpretation
m = 3
n = 3
Found vector distinct from meet with same interpretation
m = 3
n = 4


 11%|█         | 12/111 [00:00<00:04, 22.36it/s]

...

Found vector distinct from meet with same interpretation
m = 4
n = 79


 99%|█████████▉| 110/111 [04:32<00:10, 10.36s/it]

Found vector distinct from meet with same interpretation
m = 4
n = 80


100%|██████████| 111/111 [04:42<00:00,  2.54s/it]

Found vector distinct from meet with same interpretation
|models where meet has a synonym| = 111
|models where meet is unique| = 0
|models where no meet exists| = 0


In [64]:
def findAmbiguousObjectSet(numFeatures, objectSetSize, solver=None):
  if solver is None:
    solver = z3.Solver()

#   solver.set("sat.cardinality.solver", False)
  
  defineds_obj = [[z3.Bool(f"obj_{i}_def_{j}") for j in range(numFeatures)]
                  for i in range(objectSetSize)]
  valences_obj = [[z3.Bool(f"obj_{i}_val_{j}") for j in range(numFeatures)]
                  for i in range(objectSetSize)]
  
  #assert all objects have distinct feature vectors
  solver.add([distinct(i, j, defineds_obj, valences_obj) 
              for i in range(objectSetSize) for j in range(objectSetSize)])
  
  #define what's in the subset and therefore
  # - what the meet of the subset is
  # - properties of the meet that we need for subsequent constraints
  inSubset     = [z3.Bool(f"inSubset_{i}")    for i in range(objectSetSize)]
  defined_meet = [z3.Bool(f"meet_def_{j}")    for j in range(numFeatures)]
  valence_meet = [z3.Bool(f"meet_val_{j}")    for j in range(numFeatures)]
  solver.add(is_meet(defined_meet, valence_meet, inSubset, defineds_obj, valences_obj))
  meet_interp  = [z3.Bool(f"meet_interp_{i}") for i in range(objectSetSize)]
  solver.add(z3.And([z3.Implies(inSubset[i],
                                z3.And(is_le(defined_meet, valence_meet, defineds_obj[i], valences_obj[i]),
                                       meet_interp[i]))
                     for i in range(objectSetSize)]))
#   solver.add(z3.And([meet_interp[i] == z3.Implies(inSubset[i],
#                                                   is_le(defined_meet, valence_meet, defineds_obj[i], valences_obj[i]))
#                      for i in range(objectSetSize)]))
#FIXME remove
#   solver.add(z3.And([z3.And(z3.Implies(is_le(defined_meet, valence_meet, defineds_obj[i], valences_obj[i]),
#                                        meet_interp[i]),
#                             z3.Implies(z3.Not(is_le(defined_meet, valence_meet, defineds_obj[i], valences_obj[i])),
#                                        z3.Not(meet_interp[i])))
#                      for i in range(objectSetSize)]))

  
  #define an alternative feature vector
  # - assert it is distinct from the meet
  # - assert it has the same interpretation as the meet
  
  defined_alt = [z3.Bool(f"alt_def_{j}")    for j in range(numFeatures)]
  valence_alt = [z3.Bool(f"alt_val_{j}")    for j in range(numFeatures)]
  solver.add(z3.Not(eq_Booleans(defined_meet, valence_meet, defined_alt, valence_alt)))
  
  alt_interp  = [z3.Bool(f"alt_interp_{i}") for i in range(objectSetSize)]
  solver.add(z3.And([meet_interp[i] == alt_interp[i]
                     for i in range(objectSetSize)]))
#FIXME remove  
#   solver.add(z3.And([z3.Implies(meet_interp[i], alt_interp[i]),
#                      z3.Implies(z3.Not(meet_interp[i]), z3.Not(alt_interp[i])),
#                      z3.Implies(alt_interp[i], meet_interp[i]),
#                      z3.Implies(z3.Not(alt_interp[i]), z3.Not(meet_interp[i]))
#                    ]))
  
  if solver.check() == z3.sat:
    print("Model found.")
    return solver, solver.model(), defineds_obj, valences_obj, inSubset, defined_meet, valence_meet, meet_interp, defined_alt, valence_alt, alt_interp
  else:
    print("No model found.")
    return solver


# should I define a more generic "three place" (six place) predicate for meet? (in cell above?) 
def is_meet(defined_meet, valence_meet, characteristicVector_subset, defineds_obj, valences_obj):
  numFeatures   = len(defined_meet)
  objectSetSize = len(defineds_obj)
  
  # for every index j
  # if every object i in the subset is defined at j and has valence true
  # then
  #  the meet vector is defined at index j
  #  the meet vector has valence true at index j
  
  # for every index j
  # if every object i in the subset is defined at j and has valence false
  # then
  #  the meet vector is defined at index j
  #  the meet vector has valence false at index j
  
  def allDefinedAtAndWithValence(j, v):
    if v:
      return z3.And([z3.Implies(characteristicVector_subset[i],
                                z3.And([defineds_obj[i][j], valences_obj[i][j]])) 
                     for i in range(objectSetSize)])
    else:
      return z3.And([z3.Implies(characteristicVector_subset[i],
                                z3.And([defineds_obj[i][j], z3.Not(valences_obj[i][j])])) 
                     for i in range(objectSetSize)])
  
  valenceUndefinedConstraints = [z3.Not(z3.And(allDefinedAtAndWithValence(j, True ),
                                               allDefinedAtAndWithValence(j, False))) == z3.Not(defined_meet[j])
                                 for j in range(numFeatures)]
  valenceTrueConstraints      = [allDefinedAtAndWithValence(j, True ) == z3.And([defined_meet[j],        valence_meet[j]])
                                 for j in range(numFeatures)]
  valenceFalseConstraints     = [allDefinedAtAndWithValence(j, False) == z3.And([defined_meet[j], z3.Not(valence_meet[j])])
                                 for j in range(numFeatures)]
#   valenceTrueConstraints  = [z3.Implies(allDefinedAtAndWithValence(j, True),
#                                         z3.And([defined_meet[j], valence_meet[j]]))
#                              for j in range(numFeatures)]
#   valenceFalseConstraints = [z3.Implies(allDefinedAtAndWithValence(j, False),
#                                         z3.And([defined_meet[j], z3.Not(valence_meet[j])]))
#                              for j in range(numFeatures)]

  return z3.And(valenceTrueConstraints + valenceFalseConstraints)
  
def distinct(i, j, defineds, valences):
  assert len(defineds) == len(valences)
  objectSetSize = len(defineds)
  assert i < objectSetSize
  assert j < objectSetSize
  
  def_i, val_i = defineds[i], valences[i]
  def_j, val_j = defineds[j], valences[j]
  
  assert len(def_i) == len(val_i)
  assert len(def_j) == len(val_j)
  assert len(def_i) == len(def_j)
  
  numFeatures = len(def_i)
  
  # Either
  #  i == j
  # (so we actually don't want to assert that the ith and jth objects are distinct), or
  # there's at least one feature where 
  #  - one object is defined and the other is not
  #  - both objects are defined and their valence is distinct
  return z3.Or([i == j] + 
               [z3.And(def_i[x], z3.Not(def_j[x]))
                for x in range(numFeatures)] + 
               [z3.Implies(z3.And(def_i[x], def_j[x]),
                           val_i[x] != val_j[x]) 
                for x in range(numFeatures)])

In [65]:
def extract_ambiguousObjectSetModel(result, numFeatures, objectSetSize):
  solver, model, defineds_obj, valences_obj, inSubset, defined_meet, valence_meet, meet_interp, defined_alt, valence_alt, alt_interp = result
  return {"solver":solver, "model":model, "defineds_obj":defineds_obj, "valences_obj":valences_obj, "inSubset":inSubset, "defined_meet":defined_meet, "valence_meet":valence_meet, "meet_interp":meet_interp, "defined_alt":defined_alt, "valence_alt":valence_alt, "alt_interp":alt_interp}

def concretize_ambiguousObjectSetModel(extractedResult, numFeatures, objectSetSize):
  model        = extractedResult["model"]
  defineds_obj = extractedResult["defineds_obj"]
  valences_obj = extractedResult["valences_obj"]
  inSubset     = extractedResult["inSubset"]
  defined_meet = extractedResult["defined_meet"]
  valence_meet = extractedResult["valence_meet"]
  meet_interp  = extractedResult["meet_interp"]
  defined_alt  = extractedResult["defined_alt"]
  valence_alt  = extractedResult["valence_alt"]
  alt_interp   = extractedResult["alt_interp"]


  objects    = [extractFeatureVectorFromModel(model, defineds_obj[i], valences_obj[i]) for i in range(len(defineds_obj))]
  subsetVec  = [model[inSubset[i]]    for i in range(len(inSubset))                 ]
  subset     = [objects[i]            for i in range(len(objects)) if subsetVec[i]]
  meet       =  extractFeatureVectorFromModel(model, defined_meet   , valence_meet   )
  alt        =  extractFeatureVectorFromModel(model, defined_alt    , valence_alt    )
  meetInterp = [model[meet_interp[i]] for i in range(len(meet_interp))]
  altInterp  = [model[ alt_interp[i]] for i in range(len( alt_interp))]

  return {"objects":objects, "subsetVec":subsetVec, "subset":subset, "meet":meet, "alt":alt, "meetInterp":meetInterp, "altInterp":altInterp}

def report_ambiguousObjectSetModel(concretized_result):
  # objects, subsetVec, subset, meet, alt, meetInterp, altInterp = result
  objects    = concretized_result["objects"]
  subsetVec  = concretized_result["subsetVec"]
  subset     = concretized_result["subset"]
  meet       = concretized_result["meet"]
  alt        = concretized_result["alt"]
  meetInterp = concretized_result["meetInterp"]
  altInterp  = concretized_result["altInterp"]


  print(f"|objects| = {len(objects)}")
  print(f"|subset|  = {len(subset)}")
  print('---')
  print(f"subset:")
  for each in subset:
    print(each)
  print('---')
  print(f"meet vs. alt:\n{meet}\n{alt}")
  print(f"meet interp vs. alt interp:\n{meetInterp}\n{altInterp}")

In [66]:
myNumFeatures   = 3
myObjectSetSize = 10

ambiguousObjectSet_result0           = findAmbiguousObjectSet(                                                  myNumFeatures, myObjectSetSize)
ambiguousObjectSet_result0_namespace = extract_ambiguousObjectSetModel(ambiguousObjectSet_result0             , myNumFeatures, myObjectSetSize)
ambiguousObjectSet_result0_concrete  = concretize_ambiguousObjectSetModel(ambiguousObjectSet_result0_namespace, myNumFeatures, myObjectSetSize)
report_ambiguousObjectSetModel(ambiguousObjectSet_result0_concrete)

# ambiguousObjectSet_result0_models    = all_smt(ambiguousObjectSet_result0_namespace["solver"])
# print(f"|models| = {len(ambiguousObjectSet_result0_models)}")
# for eachModel in ambiguousObjectSet_result0_models:
#   namespace          = deepcopy(ambiguousObjectSet_result0_namespace)
#   namespace["model"] = eachModel
#   concrete           = concretize_ambiguousObjectSetModel(namespace, myNumFeatures, myObjectSetSize)
#   report_ambiguousObjectSetModel(concrete)

Model found.
|objects| = 10
|subset|  = 1
---
subset:
[0 - -]
---
meet vs. alt:
[0 - -]
[+ 0 +]
meet interp vs. alt interp:
[True, False, False, False, False, False, False, False, False, False]
[True, False, False, False, False, False, False, False, False, False]


In [56]:
if nextSolverState.check() == z3.sat:
  print("Next model found:")
  nextSolverState        = block_current_model(ambiguousObjectSet_result0_namespace["solver"])
  nextNamespace          = deepcopy(ambiguousObjectSet_result0_namespace)
  nextNamespace["model"] = nextSolverState.model()
  concrete               = concretize_ambiguousObjectSetModel(nextNamespace, myNumFeatures, myObjectSetSize)
  report_ambiguousObjectSetModel(concrete)

z3.z3.Solver

In [48]:
all_smt?

Signature: all_smt(s)
Docstring: <no docstring>
File:      ~/Babel/noble-cactus/<ipython-input-3-f81d737f6aff>
Type:      function


In [44]:
myNumFeatures   = 3
myObjectSetSize = 10

mySolver, myModel, myDefineds_obj, myValences_obj, myInSubset, myDefined_meet, myValence_meet, myMeet_interp, myDefined_alt, myValence_alt, myAlt_interp = findAmbiguousObjectSet(myNumFeatures, myObjectSetSize)

def extract_ambiguousObjectSetModel(result, numFeatures, objectSetSize):
  solver, model, defineds_obj, valences_obj, inSubset, defined_meet, valence_meet, meet_interp, defined_alt, valence_alt, alt_interp = result
  
  myObjects    = [extractFeatureVectorFromModel(myModel, myDefineds_obj[i], myValences_obj[i]) for i in range(len(myDefineds_obj))]
  mySubsetVec  = [myModel[myInSubset[i]]    for i in range(len(myInSubset))                 ]
  mySubset     = [myObjects[i]              for i in range(len(myObjects)) if mySubsetVec[i]]
  myMeet       =  extractFeatureVectorFromModel(myModel, myDefined_meet   , myValence_meet   )
  myAlt        =  extractFeatureVectorFromModel(myModel, myDefined_alt    , myValence_alt    )
  myMeetInterp = [myModel[myMeet_interp[i]] for i in range(len(myMeet_interp))]
  myAltInterp  = [myModel[ myAlt_interp[i]] for i in range(len( myAlt_interp))]

myObjects    = [extractFeatureVectorFromModel(myModel, myDefineds_obj[i], myValences_obj[i]) for i in range(len(myDefineds_obj))]
mySubsetVec  = [myModel[myInSubset[i]]    for i in range(len(myInSubset))                 ]
mySubset     = [myObjects[i]              for i in range(len(myObjects)) if mySubsetVec[i]]
myMeet       =  extractFeatureVectorFromModel(myModel, myDefined_meet   , myValence_meet   )
myAlt        =  extractFeatureVectorFromModel(myModel, myDefined_alt    , myValence_alt    )
myMeetInterp = [myModel[myMeet_interp[i]] for i in range(len(myMeet_interp))]
myAltInterp  = [myModel[ myAlt_interp[i]] for i in range(len( myAlt_interp))]

print(f"|objects| = {len(myObjects)}")
print(f"|subset|  = {len(mySubset)}")
print('---')
print(f"subset:")
for each in mySubset:
  print(each)
print('---')
print(f"meet vs. alt:\n{myMeet}\n{myAlt}")
print(f"meet interp vs. alt interp:\n{myMeetInterp}\n{myAltInterp}")

Model found.
|objects| = 10
|subset|  = 1
---
subset:
[- + 0 0]
---
meet vs. alt:
[- + 0 0]
[0 - 0 0]
meet interp vs. alt interp:
[True, False, False, False, False, False, False, False, False, False]
[True, False, False, False, False, False, False, False, False, False]


In [77]:
findAmbiguousObjectSet(3,2)

Model found.


([Or(True,
     And(obj_0_def_0, Not(obj_0_def_0)),
     And(obj_0_def_1, Not(obj_0_def_1)),
     And(obj_0_def_2, Not(obj_0_def_2)),
     Implies(And(obj_0_def_0, obj_0_def_0),
             obj_0_val_0 != obj_0_val_0),
     Implies(And(obj_0_def_1, obj_0_def_1),
             obj_0_val_1 != obj_0_val_1),
     Implies(And(obj_0_def_2, obj_0_def_2),
             obj_0_val_2 != obj_0_val_2)),
  Or(False,
     And(obj_0_def_0, Not(obj_1_def_0)),
     And(obj_0_def_1, Not(obj_1_def_1)),
     And(obj_0_def_2, Not(obj_1_def_2)),
     Implies(And(obj_0_def_0, obj_1_def_0),
             obj_0_val_0 != obj_1_val_0),
     Implies(And(obj_0_def_1, obj_1_def_1),
             obj_0_val_1 != obj_1_val_1),
     Implies(And(obj_0_def_2, obj_1_def_2),
             obj_0_val_2 != obj_1_val_2)),
  Or(False,
     And(obj_1_def_0, Not(obj_0_def_0)),
     And(obj_1_def_1, Not(obj_0_def_1)),
     And(obj_1_def_2, Not(obj_0_def_2)),
     Implies(And(obj_1_def_0, obj_0_def_0),
             obj_1_val_0 != obj_0_

In [48]:
def naturalClassesFor_euf_bcc(featureVectors, sortedObjectVectors, solver=None):
  if solver is None:
    solver = z3.Solver()
  
  # Setting this to true changes how z3 solves the boolean cardinality constraints further below
  # See the reference link for more information.
  #
  # Default value of this is false; I'm not sure, but I weakly expect that when this is false,
  # the underlying constraints are the same more or less as what I describe above.
  # In any case this function will work either way.
  solver.set("sat.cardinality.solver", True) 
  
  data = tuple(sorted(featureVectors, key=FeatureVector.sortKey))
  M    = len(data[0])

  
  # Calculate the meet and the properties of it that we need to find better solutions or show none 
  # exist
  meet       = grandFoldl(FeatureVector.meet)(data)
  # The meet has the best denotation we can get.
  # ("Denotation" here is a boolean tuple, one per object in sortedObjectVectors.) 
  bestInterp = FeatureVector.interpretation(meet, sortedObjectVectors)
  # The meet is the most verbose (most specified) feature vector with the denotation that it has.
  # ('spec(meet)' = number of specified features in meet.)
  worstSpec  = FeatureVector.spec(meet)
  
  
  # Now find a feature vector with the same interpretation but no worse of a spec
  defined = [z3.Bool(f"def_{i}") for i in range(M)]
  valence = [z3.Bool(f"val_{i}") for i in range(M)]
  
  # Enforce same interpretation constraint
  for (v,b) in zip(sortedObjectVectors, bestInterp):
    if b:
      solver.add(describes_euf(defined, valence, v))
    else:
      solver.add(z3.Not(describes_euf(defined, valence, v)))
  
  # Search for a model with a spec starting from 0 and ascending towards the spec of the meet
  models = []
  for maxNumDefinedFeatures in range(worstSpec + 1):
    solver.push()
    print(showVal("maxNumDefinedFeatures", f"{maxNumDefinedFeatures}"))
    solver.add(z3.AtMost(*defined, maxNumDefinedFeatures))
    
    if solver.check() == z3.sat:
      m = solver.model()
#       models.append(deepcopy(m))
      models.append(m)
      print("Found model.")
      #print(m)
      block_current_model(s)
      #return solver, models, defined, valence
    
    solver.pop()
  
  if len(models) == 0:
    print("No model (not even the trivial meet of the zero vector?!) found!") 
    return solver
  else:
    return solver, models, defined, valence
  
#   #Here we're exploiting properties that the meet has (maximality, uniqueness) in order to search 
#   # for it in part because this is a common manual form of optimizing integer variables in z3.
#   # It would be better to just define it / embed a direct calculation in its definition.
#   for minNumDefinedFeatures in range(M, -1, -1):
#     solver.push()
#     print(showVal("minNumDefinedFeatures", f"{minNumDefinedFeatures}"))
#     solver.add(z3.AtLeast(*defined, minNumDefinedFeatures))
  
#     if solver.check() == z3.sat:
#       print("Found model.")
#       m = solver.model()
#       return solver, m, defined, valence
    
#     solver.pop()
#   print("No model found.")
#   return solver
  

def describes_euf(defined, valence, featureVector):
  # \forall i, defined[i] -> valence[i] == featureVector.value[i].valence()
  return z3.And([z3.Or([z3.Not(d), v == x.valence()])
                 for d,v,x in zip(defined, valence, featureVector.value)])

# def extractFeatureVectorFromModel_asBooleans(model, defined, valence):
#   return (tuple(model[defined[i]] for i in range(len(defined))),
#           tuple(model[valence[i]] for i in range(len(valence))))

# def extractFeatureVectorFromModel(model, defined, valence):
#   concrete_defined, concrete_valence = extractFeatureVectorFromModel_asBooleans(model, defined, valence)
#   return FeatureVector.fromBooleans(concrete_defined, concrete_valence)

In [49]:
for each in S:
  print(each)

[+ - - - 0 - + - + 0]
[0 0 0 0 0 - + - - -]
[- - - + + + + - - 0]
[- 0 - + - + + - 0 +]
[0 + + + 0 0 + - 0 0]
[+ + - 0 + - + - + 0]
[0 0 - 0 + - + - 0 0]
[+ - + + + + + - - -]


In [50]:
solver_NC_EUF_BCC, models_NC_EUF_BCC, def_NC_EUF_BCC, val_NC_EUF_BCC = naturalClassFor_euf_bcc(S, O)

print(extractFeatureVectorFromModel(models_NC_EUF_BCC[0], def_NC_EUF_BCC, val_NC_EUF_BCC))

#models_NC_EUF_BCC

maxNumDefinedFeatures = 0
maxNumDefinedFeatures = 1
maxNumDefinedFeatures = 2
Found model.
[0 0 0 0 0 0 + - 0 0]


In [34]:
naturalClassFor_euf_bcc(S, O)

maxNumDefinedFeatures = 0
maxNumDefinedFeatures = 1
Found model.


([Not(And(Or(Not(def_0), False),
          Or(Not(def_1), False),
          Or(Not(def_2), False))),
  And(Or(Not(def_0), False),
      Or(Not(def_1), val_1 == True),
      Or(Not(def_2), False)),
  Not(And(Or(Not(def_0), False),
          Or(Not(def_1), False),
          Or(Not(def_2), val_2 == True))),
  Not(And(Or(Not(def_0), False),
          Or(Not(def_1), val_1 == False),
          Or(Not(def_2), val_2 == False))),
  Not(And(Or(Not(def_0), val_0 == True),
          Or(Not(def_1), False),
          Or(Not(def_2), val_2 == True))),
  And(Or(Not(def_0), val_0 == True),
      Or(Not(def_1), val_1 == True),
      Or(Not(def_2), False)),
  Not(And(Or(Not(def_0), False),
          Or(Not(def_1), val_1 == False),
          Or(Not(def_2), val_2 == True))),
  Not(And(Or(Not(def_0), val_0 == True),
          Or(Not(def_1), val_1 == False),
          Or(Not(def_2), False))),
  Not(And(Or(Not(def_0), val_0 == False),
          Or(Not(def_1), False),
          Or(Not(def_2), val_2 == False))),

In [95]:
naturalClassFor_euf_bcc(S)

minNumDefinedFeatures = 3
minNumDefinedFeatures = 2
minNumDefinedFeatures = 1
Found model.


([And(Or(Not(def_0), val_0 == True),
      Or(Not(def_1), val_1 == True),
      Or(Not(def_2), val_2 == False)),
  And(Or(Not(def_0), val_0 == False),
      Or(Not(def_1), val_1 == True),
      Or(Not(def_2), val_2 == False)),
  And(Or(Not(def_0), val_0 == True),
      Or(Not(def_1), val_1 == True),
      Or(Not(def_2), val_2 == True)),
  at-least(def_0, def_1, def_2)],
 [def_0 = False, val_1 = True, def_1 = True, def_2 = False],
 [def_0, def_1, def_2],
 [val_0, val_1, val_2])

In [94]:
solver_NC_EUF_BCC, models_NC_EUF_BCC, def_NC_EUF_BCC, val_NC_EUF_BCC = naturalClassFor_euf_bcc(S)

models_NC_EUF_BCC[0].decls()
# models_NC_EUF_BCC


'---'
concrete_def_NC_EUF_BCC = tuple([model_NC_EUF_BCC[def_NC_EUF_BCC[i]] for i in range(len(def_NC_EUF_BCC))])
concrete_val_NC_EUF_BCC = tuple([model_NC_EUF_BCC[val_NC_EUF_BCC[i]] for i in range(len(val_NC_EUF_BCC))])
concrete_def_NC_EUF_BCC, concrete_val_NC_EUF_BCC

minNumDefinedFeatures = 3
minNumDefinedFeatures = 2
minNumDefinedFeatures = 1
Found model.


[def_0, val_1, def_1, def_2]

[def_0 = False, val_1 = True, def_1 = True, def_2 = False]

'---'

((False, True, False), (None, True, None))

Success!

## Bitvectors

See [this](http://theory.stanford.edu/~nikolaj/programmingz3.html#sec-bit-vectors) for more on the theory of bitvectors in z3.

As a representation, 
 1. Bitvectors are a bit of a ball-of-mud datatype that can be easier to work with and manipulate than encodings using alternative theories or combinations of them.
 2. Solving sets of constraints on them (compared to say, using the more general theory of arrays) may be more effective or easier to make efficient.

In [ ]:
def naturalClassFor_bv(featureVectors, solver=None):
  if solver is None:
    solver = z3.Solver()
  
  data = tuple(sorted(featureVectors, key=FeatureVector.sortKey))
  M    = len(data[0])
  
  defined = z3.BitVec('def', M)
  valence = z3.BitVec('val', M)
  
  for datum in data:
    datumDefined, datumValence = datum.toBools(zeroValue=False)
    solver.add(~defined | (valence == datumValence) ) #how can/will z3 compare a symbolic bitvector to a list of Booleans?
#     solver.add(describes_bv(defined, valence, datum))
  
  for minNumDefinedFeatures in range(M, -1, -1):
    solver.push()
    print(showVal("minNumDefinedFeatures", f"{minNumDefinedFeatures}"))
    
    solver.add(z3.AtLeast(*defined, minNumDefinedFeatures))
  
    if solver.check() == z3.sat:
      print("Found model.")
      m = solver.model()
      return solver, m, defined, valence
    
    solver.pop()
  print("No model found.")
  return solver
  

# Witness the ball-of-mud-ness of bitvectors in action:
# Suppose we have a length 3 bit vector m = 101. The expression
#   m >>> 1
# denotes the bitvector 
#   010
# that results from shifting the values of m 1 place rightward ("(logical, as opposed to arithmetic)
# bitshift right"), with the rightmost value of m (i.e. the least significant bit, here the 
# rightmost "1" in "101") "falling off" and the leftmost value of m being replaced by a 0. 
#
# Significance:
# If we want to check if the ith bit of m is a 1, we can take the elementwise `and` of the m-length 
# bitvector corresponding to the unsigned integer for 1 with `m >>> i`:
#   isIthBitofMaOne = (m >>> i) & 1
# (Yes, we are taking advantage of the ability to cast bit vectors to other types.)
#
# Takeaway:
# We can extend this to use in z3 by creating m m-length bitvectors (each representing information
# about one of the m indices of our symbolic bit vector) and then a final one representing their 
# sum.
def countOnesInSymbolicBV(M: int, symbolicBitVector, solver):
  v, s             = symbolicBitVector, solver
  prefix           = f"{hash(hash(M), hash(symbolicBitVector), hash(solver))}"
  counts           = [z3.BitVec(f'{prefix}_{i}') for i in range(M)]
  countConstraints = z3.And([counts[i] == z3.LShR(v, i) & 1 for i in range(M)])
  sumOfCounts      = z3.BitVec(f'{prefix}_f')
  sumConstraint    = sumOfCounts == z3.Sum(counts)
  
  s.add(countConstraints)
  s.add(sumConstraint)
  
  if s.check() == z3.sat:
    return s, m, counts, sumOfCounts
  print("No model found(!) for counting 1s...")
  return None
    

def describes_bv(defined, valence, featureVector):
  # \forall i, defined[i] -> valence[i] == featureVector.value[i].valence()
#   return z3. #FIXME
  return z3.And([z3.Or([z3.Not(d), v == x.valence()])
                 for d,v,x in zip(defined, valence, featureVector.value)])


In [97]:
z3.Sum?

Signature: z3.Sum(*args)
Docstring:
Create the sum of the Z3 expressions.

>>> a, b, c = Ints('a b c')
>>> Sum(a, b, c)
a + b + c
>>> Sum([a, b, c])
a + b + c
>>> A = IntVector('a', 5)
>>> Sum(A)
a__0 + a__1 + a__2 + a__3 + a__4
File:      /nix/store/ymqg9qkhmqcqvjjm1kw3n6clsf2s4gym-python3-3.8.9-env/lib/python3.8/site-packages/z3/z3.py
Type:      function


In [50]:
z3.LShR?

Signature: z3.LShR(a, b)
Docstring:
Create the Z3 expression logical right shift.

Use the operator >> for the arithmetical right shift.

>>> x, y = BitVecs('x y', 32)
>>> LShR(x, y)
LShR(x, y)
>>> (x >> y).sexpr()
'(bvashr x y)'
>>> LShR(x, y).sexpr()
'(bvlshr x y)'
>>> BitVecVal(4, 3)
4
>>> BitVecVal(4, 3).as_signed_long()
-4
>>> simplify(BitVecVal(4, 3) >> 1).as_signed_long()
-2
>>> simplify(BitVecVal(4, 3) >> 1)
6
>>> simplify(LShR(BitVecVal(4, 3), 1))
2
>>> simplify(BitVecVal(2, 3) >> 1)
1
>>> simplify(LShR(BitVecVal(2, 3), 1))
1
File:      /nix/store/ymqg9qkhmqcqvjjm1kw3n6clsf2s4gym-python3-3.8.9-env/lib/python3.8/site-packages/z3/z3.py
Type:      function


In [101]:
def g(M):
  s = z3.Solver()
  a = z3.BitVec('a', M)
  b = z3.BitVec('b', M)
  c = z3.BitVec('c', M)
  s.add(a == 2)
  s.add(b == 1)
  s.add(c == z3.Sum([a,b])) # "z3.Sum(a,b)" also works fine
  if s.check() == z3.sat:
    return s, s.model()
  print("No model found.")
  return None

In [102]:
sg, mg = g(3)

In [103]:
mg

[c = 3, b = 1, a = 2]

In [55]:
def f(M):
  s = z3.Solver()
  a = z3.BitVec('a', M)
  b = z3.BitVec('b', M)
  c = z3.BitVec('c', M)
  s.add(a == 0)
  s.add(b == 1)
  #s.add(b == c)
  s.add(c == z3.LShR(b, 1))
  if s.check() == z3.sat:
    return s.model(), s
  print("No model found")
  return None

In [56]:
mf, sf = f(3)

In [57]:
mf

[c = 0, b = 1, a = 0]

In [49]:
foo = 2
foo >>= 1
foo

1

## EUF + linear arithmetic

In [29]:
# create a function that adds the right number of variables to a given solver instance to represent a single feature vector with a given number of features

# create a function that adds constraints to the solver reflecting feature vectors that are hypothesized to be in a given natural class

# "" but for rules

## EUF (+ linear real arithmetic?) + optimization

## Special order solvers

 - See [this link on z3 and solvers for special relations](http://theory.stanford.edu/~nikolaj/programmingz3.html#sec-special-relations).

## (Constrained) Horn clauses

### What are Horn clauses?

Horn clauses are a generally important "normal form" in computational logic; perhaps one of the most common places many first encounter Horn clauses is the logical programming language [Prolog](https://en.wikipedia.org/wiki/Prolog), usually in a computer science course.

A propositional formula is a Horn clause iff it is a disjunction of literals with at most one positive literal:
$$\neg p(x) \lor \neg q(x) \lor r(x)\equiv p(x) \lor q(x) \rightarrow r(x)$$
A Horn clause with exactly one positive literal is called a *definite* Horn clause, the positive literal is called the "head", and the clause is commonly written in implication form with the head at the right:
$$r(x) \leftarrow p(x) \lor q(x)$$



### What are the implications for use in an SMT solver?

1. **Definitely giving up decidability.** Most of the theories in an SMT solver that is sitting atop a SAT solver have decidable satisfiability problems (at least in isolation; some combinations of theories are also decidable). Horn clauses are more expressive, but finding a solution for a set of Horn clauses is in general undecidable.

2. **A gain in expressivity useful for program verification and synthesis.** Nevertheless, Horn clauses are useful enough and other theories are sufficiently inexpressive, finicky, and brittle for program synthesis that many of those involved in the development of z3 think that constrained Horn clauses are a better vehicle for program verification (and by extension, closely related synthesis techniques).

 1. See [this link on z3 and the Horn Clause Solver](http://theory.stanford.edu/~nikolaj/programmingz3.html#sec-horn-clause-solver).
 2. See FIXME link [this paper on constrained Horn clauses for program verification](FIXME) (by some of the researchers and developers behind `z3` - and `Lean`, I think).
 3. See FIXME link [this paper on semantically-guided synthesis](FIXME). 

## (First-order) ADTs

 - See [this link on z3 and algebraic datatypes](http://theory.stanford.edu/~nikolaj/programmingz3.html#sec-algebraic-datatypes).

(It doesn't really make sense to jump to tree-like structures for anything involving SPE-style feature vectors or operations, other than to encode syntax symbolically. In this case though, we can still encode something closer to a perfectly fine semantics than this.)